In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '../terrace')

%load_ext autoreload
%autoreload 2

In [2]:
from tqdm import tqdm
from common.cfg_utils import get_config
from datasets.make_dataset import *
from terrace.batch import *
from terrace.type_data import *
# cfg = get_config("../configs", "regression")
# loader = make_dataloader(cfg, "val")

In [5]:
make_batch_td((TensorTD(tuple()), ))

(TypeData[Tensor[(B)]],)

In [11]:
t = torch.zeros((10,))
torch.stack([t,t]).shape

torch.Size([2, 10])

In [3]:
i = iter(batch_td.rec.ndata)
i
for act in batch_td.rec.ndata:
    print(act)

TypeData[ProtResidueNode[coord=Tensor[(3)], cat_feat=Tensor[(1), dtype=int64, max_values=[38]], scal_feat=Tensor[(0)]]]


In [2]:
import torch
from tqdm import tqdm
from routines.routine import Routine
from common.cfg_utils import get_config

cfg = get_config("../configs")
# routine = Routine(cfg)

In [24]:
cfg.data.use_rec

True

In [51]:
from datasets.bigbind_act import BigBindActDataset
cfg.data.use_lig = False
cfg.data.cache = False
cfg.data.rec_graph.only_pocket = True
dataset = BigBindActDataset(cfg, "val")
dataset[10000].lig.ndata.cat_feat.shape

torch.Size([28, 5])

In [46]:
from datasets.graphs.plot_graph import plot_graph
# plot_graph(rec)
rec.ndata.cat_feat

tensor([[18],
        [10],
        [17],
        [ 0],
        [18],
        [18],
        [18],
        [17],
        [ 4],
        [ 7],
        [10],
        [16],
        [12],
        [13],
        [15],
        [14],
        [16],
        [ 1],
        [ 4],
        [12],
        [ 9],
        [12],
        [11],
        [17],
        [13],
        [14],
        [ 9],
        [ 7],
        [13]])